# AmpliGraph

## 安装

### Prerequisites

* Linux, macOS, Windows
* Python ≥ 3.8

### 创建虚拟环境

为了为安装AmpliGraph准备一个虚拟环境，任何选项都可以；这里我们将提供使用`Conda`的指令。

```
conda create --name ampligraph python=3.8
source activate ampligraph
```

完成后，我们可以继续安装TensorFlow 2，可以通过`pip`或`conda`进行安装。

```
pip install "tensorflow==2.9.0"

or 

conda install "tensorflow==2.9.0"
```

#### 为Mac OS M1芯片安装TensorFlow 2

在Mac OS上安装搭载Apple芯片的TensorFlow 2时，我们建议使用conda环境。

```
conda create --name ampligraph python=3.8
source activate ampligraph
```

创建并激活虚拟环境后，运行以下命令安装Tensorflow。

```
conda install -c apple tensorflow-deps
pip install --user tensorflow-macos==2.9.0
pip install --user tensorflow-metal==0.6
```

如果安装出现问题或需要进一步了解，请参考官方Apple开发者网站上的[Tensorflow插件页面](https://developer.apple.com/metal/tensorflow-plugin/)。

### 安装AmpliGraph

一旦Tensorflow安装完成，我们可以继续安装AmpliGraph。

要安装最新稳定版本，请使用pip：

```
pip install ampligraph
```

要检查安装情况，请运行以下命令：

```python
>>> import ampligraph
>>> ampligraph.__version__
'2.1.0'
```

如果您想要最新的开发版本，可以从[GitHub](https://github.com/Accenture/AmpliGraph.git)克隆存储库，从源代码安装AmpliGraph并切换到`develop`分支。这样，您的本地工作副本将位于`develop`分支上的最新提交。


```
git clone https://github.com/Accenture/AmpliGraph.git
cd AmpliGraph
git checkout develop
pip install -e .
```
请注意，上面的代码段以可编辑模式（`-e`）安装库。

要检查安装情况，请运行以下命令：

```python
>>> import ampligraph
>>> ampligraph.__version__
'2.1-dev'
```


在这个示例中，我们展示如何加载基准数据集，如何在其上训练模型以及我们可以使用哪些不同类型的评估协议。

In [6]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

## Load the dataset

In [7]:
import ampligraph
# Benchmark datasets are under ampligraph.datasets module
from ampligraph.datasets import load_fb15k_237
# load fb15k-237 dataset
dataset = load_fb15k_237()

## Train the model

In [8]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# you can continue training from where you left after restoring the model
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./transe_train_logs')

# create the model with transe scoring function
model = ScoringBasedEmbeddingModel(eta=5,
                                   k=300,
                                   scoring_type='TransE')

# you can either use optimizers/regularizers/loss/initializers with default values or you can 
# import it and customize the hyperparameters and pass it to compile

# Let's create an adam optimizer with customized learning rate =0.005
adam = tf.keras.optimizers.Adam(learning_rate=0.005)
# Let's compile the model with self_advarsarial loss of default parameters
model.compile(optimizer=adam, loss='self_adversarial')

# fit the model to data.
model.fit(dataset['train'],
             batch_size=10000,
             epochs=10,
             callbacks=[tensorboard_callback])

# the training can be visualised using the following command:
# tensorboard --logdir='./transe_train_logs' --port=8891 
# open the browser and go to the following URL: http://127.0.0.1:8891/

Epoch 1/10
29/29 [==============================] - 7s 233ms/step - loss: 36773.3906
Epoch 2/10
29/29 [==============================] - 6s 215ms/step - loss: 22626.3574
Epoch 3/10
29/29 [==============================] - 6s 202ms/step - loss: 17343.5254
Epoch 4/10
29/29 [==============================] - 6s 198ms/step - loss: 14640.1602
Epoch 5/10
29/29 [==============================] - 6s 223ms/step - loss: 13013.9121
Epoch 6/10
29/29 [==============================] - 6s 202ms/step - loss: 11937.1406
Epoch 7/10
29/29 [==============================] - 6s 194ms/step - loss: 11176.6133
Epoch 8/10
29/29 [==============================] - 6s 213ms/step - loss: 10612.2920
Epoch 9/10
29/29 [==============================] - 6s 196ms/step - loss: 10177.2930
Epoch 10/10
29/29 [==============================] - 6s 200ms/step - loss: 9833.4297


## Predict scores

In [9]:
pred = model.predict(dataset['test'][:5], 
                       batch_size=100)
pred

array([-2.266111 , -2.095364 , -2.289988 , -3.8879156, -4.619501 ],
      dtype=float32)

## Evaluate the model (without filter)

### 主语和宾语（s，o）破坏
这种评估协议包括通过破坏现有三元组的主语和宾语来创建损坏的三元组。这被认为是知识图嵌入模型文献中的标准评估协议。

In [10]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'],     # test set
                       batch_size=100,      # evaluation batch size
                       corrupt_side='s,o'   # sides to corrupt for scoring and ranking
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 46s 223ms/step
MR: 472.3101575496624
MRR: 0.08900155667769008
hits@1: 0.0
hits@10: 0.2428319796457579


### 宾语破坏生成负样本
通过仅更改三元组的宾语来生成损坏的三元组。

In [11]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], 
                       batch_size=100, 
                       corrupt_side='o' # corrupt only the object
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 25s 123ms/step
MR: 268.11287797240436
MRR: 0.12974471587250763
hits@1: 0.0
hits@10: 0.3512085331245719


In [12]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], 
                       batch_size=100, 
                       corrupt_side='s' # corrupt only the subject
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 27s 129ms/step
MR: 676.5074371269204
MRR: 0.048258397482872535
hits@1: 0.0
hits@10: 0.13445542616694392


## 使用过滤器进行评估
在生成的损坏三元组中，指定在过滤器内的三元组将被移除，以避免产生错误的负例。

In [13]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], 
                       batch_size=100, 
                       corrupt_side='s,o', # corrupt both subject and object
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 929s 5s/step
MR: 364.22042274195127
MRR: 0.19061439893688528
hits@1: 0.12329973578628045
hits@10: 0.32199823857520304


## 使用实体子集进行破坏评估
根据所选择的评估策略，指定用于破坏的实体子集，可以是主语、宾语或两者。请注意，尽管这不是标准的评估协议，但使用实体子集可以生成更有意义的破坏，并且还可以大大减少由于在知识图中的所有实体中抽样破坏而导致的计算开销。

In [14]:
# Let's get all the month present in training
months = set(dataset['train'][
    dataset['train'][:, 1] == 
        '/travel/travel_destination/climate./travel/travel_destination_monthly_climate/month'][:, 2])
len(months)

12

In [15]:
# consider we are evaluating the below test set which is specific to one predicate
# This predicate tells the best time of the year(o) to visit a destination (s)
dest_month_test_triples = dataset['test'][
    dataset['test'][:, 1] ==
        '/travel/travel_destination/climate./travel/travel_destination_monthly_climate/month']

In [16]:
# Let's say we want to evaluate this test set by corrupting the object with only months.
# we can pass the months as entities_subset and generate corruptions only using this subset 
# instead of all entities in the graph
# This approach is very useful when the graph size is big and/or 
# when our hypothesis belongs to a specific predicate type
# When graph size is big we can randomly sample fixed number of small subset of entities and use it as corruption

# evaluate on the test set
ranks = model.evaluate(dest_month_test_triples, 
                       batch_size=100, 
                       corrupt_side='o', # corrupt only the object
                       entities_subset=months,
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

2/2 [==============================] - 2s 1s/step
MR: 1.0833333333333333
MRR: 0.9861111111111112
hits@1: 0.9833333333333333
hits@10: 1.0


## Visualize the embeddings

In [17]:
from ampligraph.utils import create_tensorboard_visualizations

In [18]:
create_tensorboard_visualizations(model, 
                                  entities_subset=['/m/027rn', '/m/06cx9', '/m/017dcd', '/m/06v8s0', '/m/07s9rl0'], 
                                  labels=['ent1', 'ent2', 'ent3', 'ent4', 'ent5'],
                                  loc = './selected_subset_embeddings_vis')


In [ ]:
create_tensorboard_visualizations(model, 
                                  entities_subset='all',
                                  loc = './full_embeddings_vis')

# the embeddings can be visualised uncommenting the following command:
# ! tensorboard --logdir='./full_embeddings_vis' --port=8891 
# open the browser and go to the following URL: http://127.0.0.1:8891/#projector

# 提前停止
以下示例展示了如何在训练模型时使用提前停止以及如何创建检查点。

In [2]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [4]:
import ampligraph
# Benchmark datasets are under ampligraph.datasets module
from ampligraph.datasets import load_fb15k_237
# load fb15k-237 dataset
dataset = load_fb15k_237()

In [6]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# create the model with transe scoring function
model = ScoringBasedEmbeddingModel(eta=1, 
                                     k=10,
                                     scoring_type='TransE')


# compile the model with loss and optimizer
model.compile(optimizer='adam', loss='multiclass_nll')

# Use this for checkpoints at regular intervals
#checkpoint = tf.keras.callbacks.ModelCheckpoint('./chkpt_transe', monitor='val_mrr', verbose=1, save_best_only=True, mode='min')

# Use this for early stopping
checkpoint = tf.keras.callbacks.EarlyStopping(monitor="val_mrr",            # which metrics to monitor
                                              patience=3,                   # If the monitored metric doesnt improve 
                                                                            # for these many checks the model early stops
                                              verbose=1,                    # verbosity
                                              mode="max",                   # how to compare the monitored metrics. 
                                                                            # max - means higher is better
                                              restore_best_weights=True)    # restore the weights with best value

dataset = load_fb15k_237()

model.fit(dataset['train'],
          batch_size=10000,
          epochs=100,
          validation_freq=2,                            # Epochs to elapse before next evaluation
          validation_batch_size=100,                    
          validation_burn_in=1,                         # Epoch to start the validation process
          validation_data = dataset['valid'][::100],
          callbacks=[checkpoint])                       # Pass the callback to the fit function


Epoch 1/100
29/29 [==============================] - 3s 94ms/step - loss: 6622.9019
Epoch 2/100
29/29 [==============================] - ETA: 0s - loss: 6493.6064

2023-02-08 13:21:22.984591: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: RaggedFromRowLengths/RowPartitionFromRowLengths/assert_non_negative/assert_less_equal/Assert/AssertGuard/branch_executed/_8


29/29 [==============================] - 2s 81ms/step - loss: 6493.6064 - val_mrr: 0.0684 - val_mr: 3258.6080 - val_hits@1: 0.0000e+00 - val_hits@10: 0.1790 - val_hits@100: 0.3466
Epoch 3/100
29/29 [==============================] - 1s 34ms/step - loss: 6365.1802
Epoch 4/100
29/29 [==============================] - ETA: 0s - loss: 6234.0171

2023-02-08 13:21:26.191442: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: RaggedFromRowLengths/RowPartitionFromRowLengths/assert_non_negative/assert_less_equal/Assert/AssertGuard/branch_executed/_8


29/29 [==============================] - 2s 69ms/step - loss: 6234.0171 - val_mrr: 0.0809 - val_mr: 1924.0540 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2102 - val_hits@100: 0.4006
Epoch 5/100
29/29 [==============================] - 2s 59ms/step - loss: 6101.5239
Epoch 6/100
26/29 [=========================>....] - ETA: 0s - loss: 6003.9404

...

29/29 [==============================] - 2s 69ms/step - loss: 4488.2705 - val_mrr: 0.0833 - val_mr: 755.6392 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2216 - val_hits@100: 0.5199
Epoch 19/100
29/29 [==============================] - 1s 29ms/step - loss: 4393.5967
Epoch 20/100
29/29 [==============================] - ETA: 0s - loss: 4303.0571

2023-02-08 13:21:47.841319: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: RaggedFromRowLengths/RowPartitionFromRowLengths/assert_non_negative/assert_less_equal/Assert/AssertGuard/branch_executed/_8


29/29 [==============================] - 1s 48ms/step - loss: 4303.0571 - val_mrr: 0.0827 - val_mr: 739.2500 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2216 - val_hits@100: 0.5227
Epoch 21/100
29/29 [==============================] - 0s 17ms/step - loss: 4217.0073
Epoch 22/100
      2/Unknown - 0s 152ms/step===>..] - ETA: 0s - loss: 4140.2354

2023-02-08 13:21:50.470017: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: RaggedFromRowLengths/RowPartitionFromRowLengths/assert_non_negative/assert_less_equal/Assert/AssertGuard/branch_executed/_8


29/29 [==============================] - 2s 64ms/step - loss: 4134.3545 - val_mrr: 0.0839 - val_mr: 737.0227 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2244 - val_hits@100: 0.5170
Restoring model weights from the end of the best epoch: 16.
Epoch 22: early stopping


In [7]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], # test set
                       batch_size=100, # evaluation batch size
                       corrupt_side='s,o', 
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 616s 3s/step
MR: 600.0142626480086
MRR: 0.18888812773966743
hits@1: 0.1267491926803014
hits@10: 0.3130687934240141


## Save and restore

In [ ]:
# if using ModelCheckpoint then we can restore the checkpoints using restore model
# from ampligraph.utils import restore_model
# model = restore_model('chkpt_transe')

In [1]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [2]:
import ampligraph
# Benchmark datasets are under ampligraph.datasets module
from ampligraph.datasets import load_fb15k_237
# load fb15k-237 dataset
dataset = load_fb15k_237()

In [3]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# create the model with transe scoring function
model = ScoringBasedEmbeddingModel(eta=1, 
                                     k=10,
                                     scoring_type='TransE')


# compile the model with loss and optimizer
model.compile(optimizer='adam', loss='multiclass_nll')

# Use this for checkpoints at regular intervals
checkpoint = tf.keras.callbacks.ModelCheckpoint('./chkpt1_transe', monitor='val_mrr', verbose=0, 
                                                save_best_only=True, mode='min')

dataset = load_fb15k_237()

model.fit(dataset['train'],
          batch_size=10000,
          epochs=10,
          validation_freq=2,
          validation_batch_size=100,
          validation_burn_in=2,
          validation_data = dataset['valid'][::100],
          callbacks=[checkpoint])     # Pass the callback to the fit function


Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

Epoch 1/10
29/29 [==============================] - 3s 101ms/step - loss: 6625.2280
Epoch 2/10
29/29 [==============================] - 3s 92ms/step - loss: 6505.0376 - val_mrr: 0.0576 - val_mr: 3106.6989 - val_hits@1: 0.0000e+00 - val_hits@10: 0.1562 - val_hits@100: 0.3352
WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.
Epoch 3/10
29/29 [==============================] - 1s 49ms/step - loss: 6386.7886
Epoch 4/10
29/29 [==============================] - 2s 63ms/step - loss: 6267.6465 - val_mrr: 0.0673 - val_mr: 2225.0398 - val_hits@1: 0.0000e+00 - val_hits@10: 0.1903 - val_hits@100: 0.3835
Epoch 5/10
29/29 [==============================] - 1s 47ms/step - loss: 6149.4019
Epoch 6/10
29/29 [==============================] - 2s 63ms/step - loss: 6029.9697 - val_mrr: 0.0765 - val_mr: 1584.9233 - val_hits@1: 0.000

In [4]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], # test set
                       batch_size=100, # evaluation batch size
                       corrupt_side='s,o', 
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 407s 2s/step
MR: 921.3923084450533
MRR: 0.17730360323410402
hits@1: 0.12151384675604267
hits@10: 0.2851551032390645


In [5]:
from ampligraph.utils import save_model
# explictly save the model
save_model(model, 'saved_model_transE')


WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.


In [6]:
from ampligraph.utils import restore_model

# restore saved models or checkpoints
model = restore_model('saved_model_transE')

Saved model does not include a db file. Skipping.


In [7]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], # test set
                       batch_size=100, # evaluation batch size
                       corrupt_side='s,o', 
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 254s 1s/step
MR: 921.3923084450533
MRR: 0.17730360323410402
hits@1: 0.12151384675604267
hits@10: 0.2851551032390645


# 分区训练
以下示例展示了如何使用分区训练模型。

In [3]:
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import numpy as np
import ampligraph

## Train and predict scores

In [4]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

PATH_TO_DATASET = 'your/path/to/dataset/'

# create the model with transe scoring function
partitioned_model = ScoringBasedEmbeddingModel(eta=2, 
                                               k=50, 
                                               scoring_type='TransE')
partitioned_model.compile(optimizer='adam', loss='multiclass_nll')

# Here we have specified the path of the input file
# you can also load using default dataloaders load_fb15k_237() and pass numpy array inputs
partitioned_model.fit(PATH_TO_DATASET + 'wn18RR/train.txt',
                      batch_size=10000, 
                      partitioning_k=3, # set flag to partition the inputs
                      epochs=10)


_split: memory before: 848.0Bytes, after: 4.3447MB, consumed: 4.3439MB; exec time: 29.242s


/Users/alberto.bernardi/miniforge3/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2023-02-08 16:47:49.873938: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10
12/12 [==============================] - 3s 257ms/step - loss: 8690.4297
Epoch 2/10
12/12 [==============================] - 2s 159ms/step - loss: 8642.6641
Epoch 3/10
12/12 [==============================] - 2s 173ms/step - loss: 8588.0459
Epoch 4/10
12/12 [==============================] - 2s 171ms/step - loss: 8524.0557
Epoch 5/10
12/12 [==============================] - 2s 166ms/step - loss: 8454.0918
Epoch 6/10
12/12 [==============================] - 2s 167ms/step - loss: 8382.4600
Epoch 7/10
12/12 [==============================] - 2s 163ms/step - loss: 8313.3584
Epoch 8/10
12/12 [==============================] - 2s 161ms/step - loss: 8245.1660
Epoch 9/10
12/12 [==============================] - 2s 160ms/step - loss: 8180.5093
Epoch 10/10
12/12 [==============================] - 2s 155ms/step - loss: 8118.5229


In [6]:
# Unfiltered evaluation
ranks = partitioned_model.evaluate(PATH_TO_DATASET + 'wn18RR/test.txt', 
                                   batch_size=400)

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)



210 triples containing invalid keys skipped!
9/9 [==============================] - 14s 2s/step


(20079.140731874144, 0.011132840015629617, 0.0, 0.03625170998632011, 2924)

In [7]:
# Filtered evaluation
ranks = partitioned_model.evaluate(PATH_TO_DATASET + 'wn18RR/test.txt', 
                        batch_size=400,
                        corrupt_side='s,o',
                        use_filter={'train': PATH_TO_DATASET + 'wn18RR/train.txt',
                                    'valid': PATH_TO_DATASET + 'wn18RR/valid.txt',
                                    'test': PATH_TO_DATASET + 'wn18RR/test.txt'})

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)



210 triples containing invalid keys skipped!

210 triples containing invalid keys skipped!

210 triples containing invalid keys skipped!
9/9 [==============================] - 68s 8s/step


(20066.594562243503,
 0.01583735697421522,
 0.005471956224350205,
 0.038132694938440494,
 2924)

In [8]:
from ampligraph.utils import save_model
save_model(model=partitioned_model, model_name_path='./partitioned_model')

The path ./partitioned_model already exists. This save operation will overwrite the model                 at the specified path.
WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.


In [10]:
from ampligraph.utils import restore_model
model = restore_model('./partitioned_model')

Saved model does not include a db file. Skipping.


In [11]:
# Unfiltered evaluation
ranks = model.evaluate(PATH_TO_DATASET + 'wn18RR/test.txt',
                       batch_size=400)

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


210 triples containing invalid keys skipped!
9/9 [==============================] - 15s 2s/step


(20079.140731874144, 0.011132840015629617, 0.0, 0.03625170998632011, 2924)

In [12]:
ranks = model.evaluate(PATH_TO_DATASET + 'wn18RR/test.txt', 
                        batch_size=400,
                        corrupt_side='s,o',
                        use_filter={'train': PATH_TO_DATASET + 'wn18RR/train.txt',
                                    'valid': PATH_TO_DATASET + 'wn18RR/valid.txt',
                                    'test': PATH_TO_DATASET + 'wn18RR/test.txt'})

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)



210 triples containing invalid keys skipped!

210 triples containing invalid keys skipped!

210 triples containing invalid keys skipped!
9/9 [==============================] - 73s 8s/step


(20066.594562243503,
 0.01583735697421522,
 0.005471956224350205,
 0.038132694938440494,
 2924)